## Coco Annotations to VGG Labeler

In [1]:
## Imports

%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
from PIL import Image

import scipy.misc
from skimage import measure   

import time
import os
import zipfile as zf
import tarfile
import csv
import pickle
import urllib
import pandas as pd


from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import json

/home/dean/miniconda3/envs/ros-kache/lib/python3.6/site-packages/pycocotools/coco.py:49: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/dean/miniconda3/envs/ros-kache/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/dean/miniconda3/envs/ros-kache/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/dean/miniconda3/envs/ros-kache/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/dean/miniconda3/envs/ros-kache/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/dean/miniconda3/envs/r

### Helper Functions - Kache Dataset Loading

In [2]:
def maybe_download(source_url, filename):
    if not os.path.exists(IMAGES_DIRECTORY):
        os.mkdir(IMAGES_DIRECTORY)
    
    filepath = os.path.join(IMAGES_DIRECTORY, filename)
    if not os.path.exists(filepath):
        filepath, _ = urllib.request.urlretrieve(source_url, filepath)
        statinfo = os.stat(filepath)
        print('')
        print('Succesfully downloaded:', filepath, '| % d MB.' % int(statinfo.st_size*1e-6))
    return filepath

### Script Constants

In [3]:
## Dataset Parameters ##
WORKING_DIRECTORY = '/media/dean/datastore/ROAD_COCO'

COCO_DIRECTORY = '/media/dean/datastore/ROAD_COCO/darknet/data/coco/'
IMAGES_DIRECTORY = os.path.join(COCO_DIRECTORY,'images/vgg_anns/')
img_prefix = 'COCO_train2014_0000'
UNCHANGED_COCO_ANNOTATIONS_FILE = os.path.join(COCO_DIRECTORY,'annotations' , 'instances_train2014.json')

## VGG Labeler Dataset Extraction Parameters ##
PARTIAL_ANNS_CSV = os.path.join(WORKING_DIRECTORY, 'partial_detections.csv')
HEADER_ROW=['filename', 'file_size', 'file_attributes', 'region_count', 'region_id', 'region_shape_attributes', 'region_attributes']

In [4]:
example_coco = COCO(UNCHANGED_COCO_ANNOTATIONS_FILE)

loading annotations into memory...
Done (t=8.37s)
creating index...
index created!


In [ ]:
## Write Annotations to CSV

anns = []
filename = UNCHANGED_COCO_ANNOTATIONS_FILE
with open(filename, 'r') as f:
    data = json.load(f)
    ann = {}
    
    g = open(PARTIAL_ANNS_CSV, "w")
    g.write(','.join(HEADER_ROW))
    g.write('\n')
    
    # loop through each image
    urlstofilepaths = {}
    for image_data in data['images']:
        
        # Download Image if not exist
        url = image_data['flickr_url']
        fname = os.path.split(url)[-1]
        urlstofilepaths[url] = maybe_download(url, os.path.join(IMAGES_DIRECTORY, img_prefix+fname))
    
        
        # Get Image Size in Bytes
        img_file_size =  os.stat(urlstofilepaths[url]).st_size
        ann['filename'] = url
        ann['file_size'] = img_file_size
        
        # Grab all annotations corresponding to ther image
        img_annotation_ids = example_coco.getAnnIds(imgIds=image_data['id'])
        
        # Determine Region Count
        region_count = len(img_annotation_ids)
        ann['region_count'] = region_count
        
        # Loop through each annotation
        for i, annotation in enumerate(example_coco.loadAnns(img_annotation_ids)):
            ann['region_id'] = i
            cat = example_coco.loadCats([annotation['category_id']])[0]
            ann['category'] = cat['name']
            ann['x'] = annotation['bbox'][0]
            ann['y'] = annotation['bbox'][1]
            ann['width'] = annotation['bbox'][2]
            ann['height'] = annotation['bbox'][3]
        
            # Create row entry in csv table
            g.write(ann['filename'] + ',' + str(ann['file_size']) + ',"{}",' + str(ann['region_count']) + \
                    "," + str(ann['region_id']) + "," + '"{""name"":""rect"",""x"":' + str(ann['x']) + \
                    "," + '""y"":' + str(ann['y']) + "," + '""width"":' + str(ann['width']) + ','+ '""height"":' + \
                     str(ann['height']) + '}"' + "," + '"{""type"":""' + str(ann['category']) + '""}"') 
            g.write('\n')
    
